In [ ]:
import pandas as pd
import networkx as nx
from pathlib import Path
import yaml

In [ ]:
G = nx.read_graphml("assets/data/Figure_2/rulegrah_numbers.graphml")
G2 = nx.read_graphml("assets/data/Figure_2/rulegraph_tidy_fin.graphml")

In [ ]:
sup_table_1 = {}

for n in [n for n in G2.nodes]:
    x1,y1 = G.nodes[n]['x'], G.nodes[n]['y']
    x2,y2 = G2.nodes[n]['x'], G2.nodes[n]['y']
    assert x1 == x2
    assert y1 == y2
    label = G2.nodes[n]['label']
    number = G.nodes[n]['label']
    sup_table_1[number] = {'Rule Name' : label}

sup_table_1

In [ ]:
df = pd.DataFrame.from_dict(sup_table_1).T
df2 = df.reindex(columns = df.columns.tolist() + ["Tool Name", "Version", "Description", "Link", "References"])
df2

In [ ]:
rule_dir = Path("../workflow/rules/")
smk = rule_dir / "bgc_analytics.smk"

In [ ]:
import snakemake
import os
def call_snakemake(workdir, targets=None):
    return snakemake.snakemake(
        os.path.join(workdir, 'workflow/Snakefile'),
        configfiles=[os.path.join(workdir, 'config/config.yaml')],
        workdir=workdir,
        dryrun=True,
        targets=targets,
        use_conda=True,
        quiet=True,
        force_incomplete=True
        )

In [ ]:
item = call_snakemake("/datadrive/bgcflow/")

In [ ]:
bgcflow = snakemake.workflow

In [ ]:
bgcflow.

In [ ]:
from snakemake.parser import parse

In [ ]:
parse(workflow="../bgcflow/workflow/rules/antismash.smk")

In [ ]:
parse(Path("../bgcflow/workflow/rules/antismash.smk"), bgcflow)